### Notes for cleaning
Focus on lease year remaining. Lease duration and lease start have iffy values
Price = transaction price/no. of units [there are some transaction with multiple units]

### Some context:
- resale property is a completed property sold by an owner, while a subsale property involves the original buyer selling a unit before the project's completion or Temporary Occupation Permit (TOP), often to a different buyer



In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
import json
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
private_data = "../datasets/cleaned/cleaned_private.csv"

df = pd.read_csv(private_data, quotechar='"', escapechar='\\', thousands=',')